# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(os.getenv("PRICE_DATA"), "**/*.parquet"), recursive = True)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import pandas as pd

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

#Add lags for Close and Adj_close, add hi_lo_range, then assign the results to dd_feat
dd_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag_1 = x['Adj Close'].shift(1), hi_lo_range = x['High']-x['Low']), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8',
                    'hi_lo_range':'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

In [5]:
#Added for my understanding and validation
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,hi_lo_range
npartitions=90,,,,,,,,,,,,
ACIO,object,object,object,object,object,object,object,object,object,object,object,object
ACN,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
#Add returns to existing dd_feat
dd_feat = dd_feat.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [7]:
# Write your code below.
#The compute method converts dask data frame to pandas dataframe
df_ticker = dd_feat.compute()
#type(df_ticker)

#Add new feature moving_average
df_ticker["moving_average"] = (
    df_ticker.groupby("ticker")["returns"]
            .rolling(10)
            .mean()
            .reset_index(level=0,drop=True)
)


In [8]:
#Added for my understanding and validation
df_ticker.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,hi_lo_range,returns,moving_average
ticker,,,,,,,,,,,,,,
ACIO,2019-07-10,25.120001,25.160000,25.040001,25.115999,24.822029,170000.0,ACIO.csv,2019,NaN,NaN,0.119999,NaN,NaN
ACIO,2019-07-11,25.049999,25.110001,24.990000,25.110001,24.816103,385700.0,ACIO.csv,2019,25.115999,24.822029,0.120001,-0.000239,NaN
ACIO,2019-07-12,25.150000,25.219999,25.150000,25.219999,24.924814,229600.0,ACIO.csv,2019,25.110001,24.816103,0.070000,0.004381,NaN
ACIO,2019-07-15,25.160000,25.200001,25.160000,25.166000,24.871447,292600.0,ACIO.csv,2019,25.219999,24.924814,0.040001,-0.002141,NaN
ACIO,2019-07-16,25.035000,25.129999,25.035000,25.124001,24.829937,167100.0,ACIO.csv,2019,25.166000,24.871447,0.094999,-0.001669,NaN
ACIO,2019-07-17,25.129999,25.129999,24.940001,24.981001,24.688612,106300.0,ACIO.csv,2019,25.124001,24.829937,0.189999,-0.005692,NaN
ACIO,2019-07-18,25.059999,25.070000,24.975000,25.070000,24.776567,18400.0,ACIO.csv,2019,24.981001,24.688612,0.094999,0.003563,NaN
ACIO,2019-07-19,25.120001,25.180000,25.110001,25.110001,24.816103,106900.0,ACIO.csv,2019,25.070000,24.776567,0.070000,0.001596,NaN
ACIO,2019-07-22,25.040001,25.110001,25.040001,25.084000,24.790403,27200.0,ACIO.csv,2019,25.110001,24.816103,0.070000,-0.001035,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

Yes, it is necessary to convert to pandas frame since each ticker has two partition.  Dask will not automatically look backward into previous partions when calculating the moving average.
It would have been better to do it in Dask, if there is only one partion (parquet file) per ticker.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.